In [8]:
import os

print(os.listdir("/"))
print(os.listdir("/var/"))
print(os.listdir("/content/"))


['lib64', 'mnt', 'var', 'etc', 'home', 'sys', 'dev', 'usr', 'tmp', 'root', 'lib', 'opt', 'srv', 'bin', 'lib32', 'run', 'sbin', 'boot', 'proc', 'libx32', 'media', 'archive (1).zip', 'kaggle', 'content', '.dockerenv', 'datalab', 'tools', 'python-apt', 'python-apt.tar.xz', 'NGC-DL-CONTAINER-LICENSE', 'cuda-keyring_1.1-1_all.deb']
['backups', 'lock', 'mail', 'log', 'tmp', 'lib', 'opt', 'run', 'cache', 'spool', 'local', 'colab']
['.config', 'drive', 'sample_data']


In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!cp "/content/drive/My Drive/archive (1).zip" "/content/archive.zip"


In [12]:
import os
print(os.path.splitext("/content/archive.zip")[-1])


.zip


In [13]:
import zipfile

zip_path = "/content/archive.zip"
extract_path = "/content/HAR_Dataset"

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ تم فك الضغط بنجاح في:", extract_path)
except zipfile.BadZipFile:
    print("❌ الملف مش بصيغة ZIP صحيحة، جرب تعيد تحميله.")


✅ تم فك الضغط بنجاح في: /content/HAR_Dataset


In [14]:
import os


har_path = "/content/HAR_Dataset"


print(os.listdir(har_path))


['Human Action Recognition']


In [15]:
import os

har_path = "/content/HAR_Dataset/Human Action Recognition"


print(os.listdir(har_path))


['test', 'train', 'Testing_set.csv', 'Training_set.csv']


In [16]:
import pandas as pd


train_df = pd.read_csv("/content/HAR_Dataset/Human Action Recognition/Training_set.csv")
test_df = pd.read_csv("/content/HAR_Dataset/Human Action Recognition/Testing_set.csv")

print("Training Data:")
print(train_df.head())

print("\nTesting Data:")
print(test_df.head())


Training Data:
      filename         label
0  Image_1.jpg       sitting
1  Image_2.jpg  using_laptop
2  Image_3.jpg       hugging
3  Image_4.jpg      sleeping
4  Image_5.jpg  using_laptop

Testing Data:
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg


In [17]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

train_path = "/content/HAR_Dataset/Human Action Recognition/train"
test_path = "/content/HAR_Dataset/Human Action Recognition/test"


train_df = pd.read_csv("/content/HAR_Dataset/Human Action Recognition/Training_set.csv")


image_size = (64, 64)


X_train, y_train = [], []
for index, row in train_df.iterrows():
    img_path = os.path.join(train_path, row['filename'])
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, image_size)
        X_train.append(img)
        y_train.append(row['label'])


X_train = np.array(X_train) / 255.0


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train)


print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Labels: {label_encoder.classes_}")


Shape of X_train: (12600, 64, 64, 3)
Shape of y_train: (12600, 15)
Labels: ['calling' 'clapping' 'cycling' 'dancing' 'drinking' 'eating' 'fighting'
 'hugging' 'laughing' 'listening_to_music' 'running' 'sitting' 'sleeping'
 'texting' 'using_laptop']


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed


model = Sequential()

model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(1, 64, 64, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))


model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dropout(0.5))


model.add(Dense(15, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 1, 64, 64, 32)       │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 1, 32, 32, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 1, 32, 32, 64)       │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 1, 16, 16, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 1, 16, 16, 128)      │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 1, 8, 8, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_6 (TimeDistributed) │ (None, 1, 8192)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │       2,113,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 15)                  │             975 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,208,015 (8.42 MB)

 Trainable params: 2,208,015 (8.42 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
X_train = X_train.reshape((X_train.shape[0], 1, 64, 64, 3))


In [38]:
history = model.fit(X_train[:100], y_train[:100], epochs=1, batch_size=32, validation_split=0.2)



3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 413ms/step - accuracy: 0.1758 - loss: 2.5073 - val_accuracy: 0.3000 - val_loss: 2.3196


In [39]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
315/315 ━━━━━━━━━━━━━━━━━━━━ 152s 479ms/step - accuracy: 0.2032 - loss: 2.4299 - val_accuracy: 0.2615 - val_loss: 2.2500
Epoch 2/5
315/315 ━━━━━━━━━━━━━━━━━━━━ 196s 463ms/step - accuracy: 0.2618 - loss: 2.2473 - val_accuracy: 0.2885 - val_loss: 2.1778
Epoch 3/5
315/315 ━━━━━━━━━━━━━━━━━━━━ 213s 500ms/step - accuracy: 0.3178 - loss: 2.1120 - val_accuracy: 0.3278 - val_loss: 2.0587
Epoch 4/5
315/315 ━━━━━━━━━━━━━━━━━━━━ 189s 459ms/step - accuracy: 0.3718 - loss: 1.9369 - val_accuracy: 0.3393 - val_loss: 2.0371
Epoch 5/5
315/315 ━━━━━━━━━━━━━━━━━━━━ 209s 481ms/step - accuracy: 0.4166 - loss: 1.8274 - val_accuracy: 0.3587 - val_loss: 1.9849


In [40]:
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)


Epoch 1/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 144s 457ms/step - accuracy: 0.4629 - loss: 1.6813 - val_accuracy: 0.3635 - val_loss: 1.9405
Epoch 2/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 152s 484ms/step - accuracy: 0.5130 - loss: 1.5082 - val_accuracy: 0.3556 - val_loss: 1.9657
Epoch 3/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 152s 483ms/step - accuracy: 0.5521 - loss: 1.3878 - val_accuracy: 0.3754 - val_loss: 1.9516
Epoch 4/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 191s 447ms/step - accuracy: 0.6038 - loss: 1.2247 - val_accuracy: 0.3857 - val_loss: 1.9886
Epoch 5/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 142s 447ms/step - accuracy: 0.6534 - loss: 1.0829 - val_accuracy: 0.3817 - val_loss: 2.0187
Epoch 6/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 139s 442ms/step - accuracy: 0.7111 - loss: 0.9357 - val_accuracy: 0.3750 - val_loss: 2.0583
Epoch 7/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 139s 443ms/step - accuracy: 0.7436 - loss: 0.8179 - val_accuracy: 0.3726 - val_loss: 2.1121
Epoch 8/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 140s 446ms/step - accuracy: 0.7723 -

In [41]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 145s 457ms/step - accuracy: 0.8956 - loss: 0.3371 - val_accuracy: 0.3659 - val_loss: 2.5969
Epoch 2/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 198s 447ms/step - accuracy: 0.9078 - loss: 0.2926 - val_accuracy: 0.3679 - val_loss: 2.6192
Epoch 3/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 140s 442ms/step - accuracy: 0.9044 - loss: 0.2981 - val_accuracy: 0.3484 - val_loss: 2.7272
Epoch 4/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 142s 441ms/step - accuracy: 0.8937 - loss: 0.3179 - val_accuracy: 0.3591 - val_loss: 2.7460


In [42]:
from tensorflow.keras.layers import Dropout

model.add(Dropout(0.3))  # 30% من الوحدات ستتوقف عشوائيًا أثناء التدريب


In [43]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))


In [44]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)  # أقل من القيمة الافتراضية 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)


In [46]:
X_train = np.squeeze(X_train)  # يحذف البعد الزائد تلقائيًا إذا كان 1


In [47]:
datagen.fit(X_train)


In [49]:
X_train = np.squeeze(X_train)
print(X_train.shape)  # تأكدي إنه بقى (12600, 64, 64, 3)


(12600, 64, 64, 3)


In [50]:
print(X_train.dtype)  # لازم يكون float32 أو float64


float64


In [52]:
X_train = X_train.astype('float32')  # تحويل البيانات إلى float32
datagen.fit(X_train)  # إعادة تشغيل fit


In [51]:
X_train = X_train.astype('float32') / 255.0  # تطبيع البيانات


In [53]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# إنشاء كائن التهيئة
datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.2,  # إزاحة عرضية
    height_shift_range=0.2,  # إزاحة طولية
    horizontal_flip=True,  # قلب الصورة أفقيًا
    zoom_range=0.2  # تكبير وتصغير
)

# تطبيق التحسينات على البيانات
datagen.fit(X_train)


In [55]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات 80% تدريب و 20% تحقق
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(X_train.shape, X_val.shape)  # تأكيد الأحجام


(10080, 64, 64, 3) (2520, 64, 64, 3)


In [57]:
print(X_train.shape)
print(y_train.shape)



(10080, 64, 64, 3)
(10080, 15)


In [58]:
sample_batch = next(datagen.flow(X_train, y_train, batch_size=32))
print(sample_batch[0].shape)  # شكل الصور
print(sample_batch[1].shape)  # شكل التصنيفات


(32, 64, 64, 3)
(32, 15)


In [59]:
X_train = X_train.reshape((X_train.shape[0], 1, 64, 64, 3))
X_val = X_val.reshape((X_val.shape[0], 1, 64, 64, 3))


In [62]:
model.add(Dense(15, activation='softmax'))  # 15 = عدد الفئات الصحيحة



In [23]:
print(X_train.shape)
print(y_train.shape)


(12600, 1, 64, 64, 3)
(12600, 15)


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# بناء نموذج CNN
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))  # التأكد من تطابق عدد الفئات

# تجميع الموديل
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# طباعة ملخص النموذج
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 685,135 (2.61 MB)

 Trainable params: 685,135 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
print(X_train.shape)
print(y_train.shape)



(10080, 1, 64, 64, 3)
(10080, 15)


In [66]:
X_train = X_train / 255.0


In [67]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=15)


In [68]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [70]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# إزالة البُعد الإضافي
X_train = np.squeeze(X_train)  # الآن (10080, 64, 64, 3)

# تطبيع القيم
X_train = X_train / 255.0

# التأكد من أن التصنيفات One-Hot Encoded
y_train = to_categorical(y_train, num_classes=15)

# تقسيم البيانات إلى تدريب واختبار
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# طباعة الشكل النهائي للبيانات
print(X_train.shape, y_train.shape)  # تأكيد على الأبعاد بعد التعديل



(6451, 64, 64, 3) (6451, 15, 15, 15)


In [72]:
!pip install -q tf-models-official


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [75]:
!pip install keras-cv --quiet


In [77]:
!pip install transformers


In [79]:
from transformers import ViTForImageClassification

# تحميل ViT مع ضبط عدد التصنيفات
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224", num_labels=15, ignore_mismatched_sizes=True
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [1]:
!pip install torch torchvision transformers datasets


In [3]:
import torch
import os
import pandas as pd
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor, Trainer, TrainingArguments

# تحميل النموذج وضبط عدد التصنيفات
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224", num_labels=15, ignore_mismatched_sizes=True
)

# تحميل البيانات
har_path = "/content/HAR_Dataset/Human Action Recognition"

train_df = pd.read_csv(os.path.join(har_path, "Training_set.csv"))
test_df = pd.read_csv(os.path.join(har_path, "Testing_set.csv"))

print("Training Data:")
print(train_df.head())

print("\nTesting Data:")
print(test_df.head())

# تحميل المحول الجديد للصور
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# تحميل مجموعة البيانات (تأكد من المسار الصحيح)
dataset = load_dataset(har_path)  # استبدل بالمسار الفعلي

# طباعة عينة من البيانات للتحقق
sample_image = torch.rand(3, 224, 224)  # صورة عشوائية كعينة
print("Sample Image Shape:", sample_image.shape)

# طباعة التصنيف لعينة من البيانات
sample_label = train_df.iloc[0]["label"]
print("Label:", sample_label)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Data:
      filename         label
0  Image_1.jpg       sitting
1  Image_2.jpg  using_laptop
2  Image_3.jpg       hugging
3  Image_4.jpg      sleeping
4  Image_5.jpg  using_laptop

Testing Data:
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg


Resolving data files:   0%|          | 0/12601 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5410 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Sample Image Shape: torch.Size([3, 224, 224])
Label: sitting


In [4]:
num_classes = len(train_df["label"].unique())
print("Number of Classes:", num_classes)


Number of Classes: 15


In [ ]:
import torch
import os
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import ViTFeatureExtractor, ViTForImageClassification, Trainer, TrainingArguments
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image

# 📌 تحميل البيانات
train_df = pd.read_csv("/content/HAR_Dataset/Human Action Recognition/Training_set.csv")
test_df = pd.read_csv("/content/HAR_Dataset/Human Action Recognition/Testing_set.csv")

# ✅ تحويل DataFrame إلى Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# ✅ تحميل Feature Extractor من ViT
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# ✅ معالجة البيانات وتحويل الصور
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ✅ دالة تجهيز البيانات
def preprocess_data(example):
    image_path = f"/content/HAR_Dataset/Human Action Recognition/train/{example['filename']}"
    image = Image.open(image_path).convert("RGB")  # التأكد من أنها صورة RGB
    example["pixel_values"] = transform(image)
    return example

# 🔄 تطبيق المعالجة على البيانات
train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

# ✅ تحميل نموذج ViT
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(train_df["label"].unique())  # عدد الفئات
)

# ✅ إعداد TrainingArguments بدون wandb
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # تعطيل wandb
)

# ✅ إعداد المدرب Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=feature_extractor  # ضروري عشان يعمل معالجة صحيحة
)

# 🚀 بدء التدريب
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/12600 [00:00<?, ? examples/s]

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

In [1]:
!pip install --upgrade transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [2]:
import torch
from transformers import ViTForImageClassification, ViTImageProcessor, Trainer, TrainingArguments
from datasets import load_dataset
import os

# تحميل البيانات
dataset = load_dataset("imagefolder", data_dir="/path/to/dataset")  # غيّر المسار حسب بياناتك

# تحميل المعالج (بدل ViTFeatureExtractor)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# تجهيز البيانات للـ model
def transform(example_batch):
    images = [image.convert("RGB") for image in example_batch["image"]]
    inputs = processor(images, return_tensors="pt")
    inputs["label"] = example_batch["label"]
    return inputs

dataset = dataset.map(transform, batched=True)

# تحميل الموديل
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=15,  # عدّل حسب عدد الفئات عندك
    ignore_mismatched_sizes=True
)

# إعدادات التدريب
training_args = TrainingArguments(
    output_dir="./vit_model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # لإيقاف wandb
)

# التأكد من أن البيانات مش فاضية
print(next(iter(dataset["train"])))

# تجهيز المدرب (Trainer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

# بدء التدريب
trainer.train()

EmptyDatasetError: The directory at /path/to/dataset doesn't contain any data files

In [3]:
import os
har_path = "/content/HAR_Dataset/Human Action Recognition"
print(os.listdir(har_path))  # يعرض الملفات والمجلدات داخل المسار


['test', 'train', 'Testing_set.csv', 'Training_set.csv']


In [4]:
import os
import pandas as pd
from datasets import load_dataset

# تحديد المسار الرئيسي
har_path = "/content/HAR_Dataset/Human Action Recognition"

# عرض الملفات والمجلدات داخل المجلد
print("المجلدات والملفات في HAR_Dataset:")
print(os.listdir(har_path))

# تحميل بيانات CSV
train_df = pd.read_csv(os.path.join(har_path, "Training_set.csv"))
test_df = pd.read_csv(os.path.join(har_path, "Testing_set.csv"))

print("\n🟢 Training Data:")
print(train_df.head())

print("\n🟢 Testing Data:")
print(test_df.head())

# تحميل الصور من المجلدين train و test
train_dataset = load_dataset("imagefolder", data_dir=os.path.join(har_path, "train"))
test_dataset = load_dataset("imagefolder", data_dir=os.path.join(har_path, "test"))

print("\n✅ تم تحميل بيانات الصور بنجاح!")
print(train_dataset)
print(test_dataset)


المجلدات والملفات في HAR_Dataset:
['test', 'train', 'Testing_set.csv', 'Training_set.csv']

🟢 Training Data:
      filename         label
0  Image_1.jpg       sitting
1  Image_2.jpg  using_laptop
2  Image_3.jpg       hugging
3  Image_4.jpg      sleeping
4  Image_5.jpg  using_laptop

🟢 Testing Data:
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg


Resolving data files:   0%|          | 0/12601 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/5410 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


✅ تم تحميل بيانات الصور بنجاح!
DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 12601
    })
})
DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 5410
    })
})


In [5]:
import os
import pandas as pd
from datasets import load_dataset, DatasetDict

# تحديد المسار الرئيسي
har_path = "/content/HAR_Dataset/Human Action Recognition"

# تحميل بيانات CSV
train_df = pd.read_csv(os.path.join(har_path, "Training_set.csv"))
test_df = pd.read_csv(os.path.join(har_path, "Testing_set.csv"))

# التأكد من وجود labels في الـ Training Data
if "label" not in train_df.columns:
    raise ValueError("الملف Training_set.csv لا يحتوي على عمود label!")

# تحميل الصور
train_dataset = load_dataset("imagefolder", data_dir=os.path.join(har_path, "train"))
test_dataset = load_dataset("imagefolder", data_dir=os.path.join(har_path, "test"))

# تحويل train_df إلى قاموس لربط الصور بـ labels
label_dict = {row["filename"]: row["label"] for _, row in train_df.iterrows()}

# إضافة labels للـ dataset
def add_labels(example):
    filename = os.path.basename(example["image"].filename)  # استخراج اسم الملف فقط
    example["label"] = label_dict.get(filename, "unknown")  # تعيين label بناءً على الاسم
    return example

# تطبيق الدالة على الـ dataset
train_dataset = train_dataset.map(add_labels)

# عرض البيانات بعد التعديل
print("\n✅ بعد إضافة الـ Labels:")
print(train_dataset)


Resolving data files:   0%|          | 0/12601 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5410 [00:00<?, ?it/s]

Map:   0%|          | 0/12601 [00:00<?, ? examples/s]


✅ بعد إضافة الـ Labels:
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12601
    })
})


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam


In [8]:
import numpy as np

# استخراج عدد الفئات من البيانات
num_classes = len(np.unique(train_df["label"]))  # عدد الفئات الفريدة في الداتا


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed

# إدخال Input بدل input_shape
input_layer = Input(shape=(None, 64, 64, 3))

x = TimeDistributed(Conv2D(32, (3,3), activation='relu', padding='same'))(input_layer)
x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)

x = TimeDistributed(Conv2D(64, (3,3), activation='relu', padding='same'))(x)
x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)

x = TimeDistributed(Flatten())(x)

x = LSTM(64, return_sequences=False)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)


In [10]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])


In [11]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 64, 64, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, None, 64, 64, 32)    │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_6 (TimeDistributed) │ (None, None, 32, 32, 32)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_7 (TimeDistributed) │ (None, None, 32, 32, 64)    │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_8 (TimeDistributed) │ (None, None, 16, 16, 64)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_9 (TimeDistributed) │ (None, None, 16384)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │       4,210,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,240,591 (16.18 MB)

 Trainable params: 4,240,591 (16.18 MB)

 Non-trainable params: 0 (0.00 B)